In [11]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from ipyparallel import Client
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.interfaces.fsl import ExtractROI

%matplotlib inline

In [12]:
rc = Client()
dview = rc[:]

In [13]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_cb_mappings.txt'
acq_params = home_dir + 'scripts/acqparams.txt'
os.chdir(home_dir)

num_slices = 57 #if you don't know, use nib.load and shape attribute
#get subject list
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))

In [5]:
#function which adds slices to scans for topup (which neeeds even number of slices)
def add_slice(scan):
    old_file = abspath(scan)
    new_file = abspath(scan[:-7] + '_' + str(num_slices + 1)) + '.nii.gz'
    if True:#not os.path.exists(new_file): #only run if target doesnt exist. 
        fslroi = ExtractROI(in_file=old_file, roi_file=new_file,
                            x_min=0,x_size=-1,y_min=0,y_size=-1,
                            z_min=0,z_size= (num_slices + 1))
        fslroi.run()
    return os.getpid()

In [18]:
##get list of scans to run
scans = []
for sub in sub_list:
    design_dir = os.path.exists('data/' + sub + '/design') #create design files AFTER 
    
#     if not design_dir:
#         scans.extend(glob.glob('data/' + sub + '/cal/*')) #calibration scans
#         scans.extend(glob.glob('data/' + sub + '/func/*/*.nii.gz')) #functional scans
    
    scans.extend(glob.glob('data/' + sub + '/func/sim/*.nii.gz')) #functional scans        
scans = [s for s in scans if len(s.split('/')[-1].split('_'))==2] #just scans that havent been 
scans =  ['data/fd_109/func/sim/run_2.nii.gz','data/fd_109/func/sim/run_1.nii.gz']

In [19]:
#run analysis
pool = multiprocessing.Pool(processes = 15)
pool.map(add_slice,scans)
pool.terminate()
pool.join()

In [26]:
##take a timepoint of the gradient reversal scans
def slice_scans(sub):
    identifier = str(num_slices + 1) #file identifier
    scans = glob.glob('data/' + sub + '/cal/*' + identifier + '.nii.gz') #padded calibration scans
    for scan in scans:
        old_file = abspath(scan)
        new_file = abspath(scan[:-7] + '_slice.nii.gz')
        if not os.path.exists(new_file):
            fslroi = ExtractROI(in_file=old_file, roi_file=new_file,
                                t_min=1,t_size=1)
            fslroi.run()
    return

In [16]:
# pool = multiprocessing.Pool()
# pool.map(slice_scans,sub_list)
# pool.terminate()
# pool.join()

In [60]:
##merge the calibration scans
def merge_scans(sub):
    identifier = 'slice' #file identifier
    cal1 = glob.glob('data/' + sub + '/cal/cal1*' + identifier + '*') #padded calibration scans
    cal2 = glob.glob('data/' + sub + '/cal/cal2*' + identifier + '*') #padded calibration scans

    if sub == 'fd_115': #extra set of scans for this subject (see notes)
        num_scans = 5
    else:
        num_scans = 4
    for i in range(1,num_scans+1):
        pe1 = [s for s in cal1 if int(s.split('/')[-1].split('_')[1]) == i][0]
        pe0 = [s for s in cal2 if int(s.split('/')[-1].split('_')[1]) == i][0]
        out = abspath('data/' + sub + '/cal/b0_both_' + str(i) + '.nii.gz')
        if not os.path.exists(out):
            merger = Merge(in_files=[pe0,pe1], merged_file=out,dimension = 't')
            merger.run()    
    return

In [17]:
# pool = multiprocessing.Pool()
# pool.map(merge_scans,sub_list)
# pool.terminate()
# pool.join()

In [7]:
#execute topup
def topup_scans(scan):
    new_file = abspath(scan[:-7] + '_topup') 
    unwarped = abspath(scan[:-7] + '_uw') 
    log = abspath(scan[:-7] + '_log') 
    field = abspath(scan[:-7] + '_field')
    if not os.path.exists(field +'.nii.gz'): #only run if not already run
        topup = TOPUP(in_file=scan, encoding_file = acq_params, out_base = new_file,
                      out_corrected = unwarped, out_logfile = log,out_field = field)
        topup.run()
    return


In [9]:
#get list of cal scans to apply topup to
scans = []
for sub in sub_list:
    design_dir = os.path.exists('data/' + sub + '/design') #create design files AFTER 
    if not design_dir:
        scans.extend(glob.glob('data/' + sub + '/cal/b0*.nii.gz')) #padded calibration scans
scans = [s for s in scans if len(s.split('/')[-1].split('_'))==3] #just take raw images
scans

[]

In [18]:
# #run analysis
# pool = multiprocessing.Pool()
# pool.map(topup_scans,scans)
# pool.terminate()
# pool.join()

In [20]:
##apply topup to functional runs. This has some experiment specific organizing
##because I acquire enough data for 4 fieldmaps throughout the experiment
def apply_topup(scan_tuple):
    #padded functional scans
    sub, scan  = scan_tuple
    out = scan[:-10] + '_fc.nii.gz' #output file name

        
    if not os.path.exists(out): #only run if it hasn't been run already
        #get exp id (sim, loc, func) and run #
        exp = scan.split('/')[3] 
        run = int(scan.split('/')[4].split('_')[1])

        if exp == 'loc': #localizers were run last
            cal_scan = 4
        elif run == 1:
            cal_scan = 1
        elif run == 2:
            cal_scan = 2
        elif run == 3:
            cal_scan = 3

        #for this subject, run2 cal scans are junk
        if sub == 'fd_101' and run == 2 :
            cal_scan = 1

        #one subject has extra cal scan for 2nd localizer run on a different day
        if sub == 'fd_115' and exp == 'loc' and run == 2:
            cal_scan = 5

        if sub == 'fd_127' and exp == 'sim' and run == 2:
            cal_scan = 2 #one is messed up

        #get correct field inputs
        base_file = abspath('data/' + sub + '/cal/b0_both_' + str(cal_scan))
        movpar = base_file + '_topup_movpar.txt'
        fieldcoef = base_file + '_topup_fieldcoef.nii.gz'

        #run motion correction
        motion_target = abspath('data/' + sub + '/cal/b0_both_' + str(cal_scan) + '_motion_target')
        cmd = ['fslroi',base_file,motion_target,'0','1']
        os.system(' '.join(cmd))
        
        m_corr = scan[:-10] + '_mc.nii.gz'
        cmd = ['mcflirt','-in',scan,'-reffile',motion_target,
               '-o',m_corr,'-mats','-plots','-rmsrel','-rmsabs']
        os.system(' '.join(cmd))
                                
        #apply topup
        applytopup = ApplyTOPUP(in_files = m_corr, encoding_file = acq_params,
                                out_corrected = out, in_index = [1], method = 'jac',
                                in_topup_movpar = movpar, in_topup_fieldcoef = fieldcoef)  
        applytopup.run()      
    return

In [21]:
#get list of scans to apply topup to (in a tuple with subject identifier for convenience)
scan_tuples = []
for sub in sub_list:
    design_dir = os.path.exists('data/' + sub + '/design') #create design files AFTER 
    if True:#not design_dir:
        scans = glob.glob('data/' + sub + '/func/*/*' + str(num_slices + 1) + '.nii.gz')
        for s in scans:
            scan_tuples.append((sub,s))

In [22]:
pool = multiprocessing.Pool(processes = 6)
pool.map(apply_topup,scan_tuples)
pool.terminate()
pool.join()

/usr/lib/python2.7/dist-packages/nipype/interfaces/base.py:397: UserWarning: Input in_topup_movpar requires inputs: in_topup_fieldcoef
  warn(msg)
/usr/lib/python2.7/dist-packages/nipype/interfaces/base.py:397: UserWarning: Input in_topup_movpar requires inputs: in_topup_fieldcoef
  warn(msg)


In [13]:
#Basic double checking based on files size. Mostly will check for processes prememptively
#killed due to memory overload
for sub in sub_list:
    scans = glob.glob('data/' + sub + '/func/*/*fc.nii.gz')
    b0 = glob.glob('data/' + sub + '/cal/*field.nii.gz')
#     if len(scans) != 8:
#         print sub
    #check b0 for file size
    for b in b0:
        file_size = np.round(os.path.getsize(b)/1000.0)
        if file_size < 4000: #less than 4 MB
            print b
    
    #check field corrected functionals for file size
    for scan in scans:
        file_size = np.round(os.path.getsize(scan)/1000000.0)
        orig_file = scan[:-10] + '.nii.gz' 
        orig_size = np.round(os.path.getsize(orig_file)/1000000.0)
        if file_size < 400: #scan less than 400 MB
            print scan
        
        if abs(file_size - orig_size) > 100: #big difference between file sizes
            print abs(file_size - orig_size)
            print scan            

In [15]:
##Specific to this experiment. Sanity check that ser/sim assignment has been done correctly by
##examining number of timepoints
for sub in sub_list:
    loc = glob.glob('data/' + sub + '/func/loc/*58*')
    sim = glob.glob('data/' + sub + '/func/sim/*58*')
    ser = glob.glob('data/' + sub + '/func/ser/*58*')
    
    for scan in loc:
        img = nib.load(scan)
        time = img.shape[3]*1.5
        if time < 300 or time > 400: #should be around 5.5 minutes
            print scan
            print time
    for scan in sim:
        img = nib.load(scan)
        time = img.shape[3]*1.5
        if time < 360 or time > 480: #should be around 7 minutes
            print scan
            print time
    for scan in ser:
        img = nib.load(scan)
        time = img.shape[3]*1.5
        if time < 450 or time > 600: #should be around 8.5 minutes
            print scan
            print time
            
#Note to self: First subject had longer scans because I was worried about stopping the scanner early


# alternative strategy using lyman

In [18]:
##merge the calibration scans
def merge_scans(sub):
    home_dir = '/home/iballard/fd/'
    cal_dir = home_dir + 'data/' + sub + '/cal'
    out_dir = cal_dir + 'lyman/'
                              
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    for i in range(1,4+1):

        out_f = out_dir + '/merged_' + str(i)
                             
        cal1 = cal_dir + 'cal1_' + str(i) + '.nii.gz'
        cal2 = cal_dir + 'cal2_' + str(i) + '.nii.gz'
        
        out_cal1 = cal_dir + 'cal1_' + str(i) + '_slice.nii.gz'
        out_cal2 = cal_dir + 'cal2_' + str(i) + '_slice.nii.gz'

        cmd = ['fslroi',cal1,out_cal1,'1','1']
        os.system(' '.join(cmd))

        cmd = ['fslroi',cal2,out_cal2,'1','1']
        os.system(' '.join(cmd))      
        
        cmd = ['fslmerge','-t',out_f,out_cal2,out_cal1]
        os.system(' '.join(cmd))
    return out_dir

In [ ]:
#run analysis
pool = multiprocessing.Pool(processes = 15)
pool.map(merge_scans,sub_list)
pool.terminate()
pool.join()

In [14]:
def create_fm(scan_tuple):
    sub, exp, run = scan_tuple

    home_dir = '/home/iballard/fd/'
    out_dir = home_dir + 'data/' + sub + '/cal/lyman/'
    data_f = home_dir + 'data/' + sub + '/func/' + exp + '/run_' + str(run) + '.nii.gz'

    if os.path.exists(data_f):
        if exp == 'loc': #localizers were run last
            cal_scan = 4
        elif run == 1:
            cal_scan = 1
        elif run == 2:
            cal_scan = 2
        elif run == 3:
            cal_scan = 3

        #for this subject, run2 cal scans are junk
        if sub == 'fd_101' and run == 2 :
            cal_scan = 1

        #one subject has extra cal scan for 2nd localizer run on a different day
        if sub == 'fd_115' and exp == 'loc' and run == 2:
            cal_scan = 5

        if sub == 'fd_127' and exp == 'sim' and run == 2:
            cal_scan = 2 #one is messed up

        #get correct field inputs
        cal_scan = out_dir + '/merged_' + str(cal_scan)
        out_cal = out_dir + '/fm_' + exp + '_' + str(run) + '_mc'
        
        if not os.path.exists(out_cal):
            #run motion correction
            out_f = out_dir + '/' + exp + str(run) + '_slice.nii.gz'
            idx = (nib.load(data_f).shape[-1] - 8)//2 #middle volume
            cmd = ['fslroi',data_f,out_f,str(idx),'1']
            os.system(' '.join(cmd))

            cmd = ['mcflirt','-in',cal_scan,'-reffile',out_f,'-out',out_cal]
            os.system(' '.join(cmd))                          
    return out_dir

In [19]:
in_tuples = []
for sub in sub_list:
    for exp in ['ser','sim']:
        for run in range(1,4):
            in_tuples.append((sub,exp,run))
    for exp in ['loc']:
        for run in range(1,3):
            in_tuples.append((sub,exp,run))

In [16]:
dview.block = True
# dview.push(dict(home_dir=home_dir))
dview.execute("import numpy as np")
dview.execute("import nibabel as nib")
with dview.sync_imports():
    import numpy
    import os
    from os.path import abspath
    import nibabel as nib
out1 = dview.map_sync(create_fm,in_tuples)

importing numpy on engine(s)
importing os on engine(s)
importing abspath from os.path on engine(s)
importing nibabel on engine(s)


In [3]:
# #run analysis
# pool = multiprocessing.Pool(processes = 15)
# pool.map(create_fm,in_tuples)
# pool.terminate()
# pool.join()
# out1

In [29]:
for scan_tuple in in_tuples:
    sub, exp, run = scan_tuple

    out_dir = os.path.abspath('data/' + sub + '/cal/lyman/')
    out_cal = out_dir + '/fm_' + exp + '_' + str(run) + '_mc.nii.gz'

    if os.path.exists(out_cal):

        tmp1 =  out_dir + '/' + sub + exp + str(run) + '_tmp1'
        tmp2 =  out_dir + '/' + sub + exp + str(run) + '_tmp2'
        tmp3 =  out_dir + '/fm_' + exp + '_' + str(run) + '_mc_pe0_pe1.nii.gz'
        
        cmd = ['fslroi',out_cal,tmp1,'0','1']
        os.system(' '.join(cmd))
        
        cmd = ['fslroi',out_cal,tmp2,'1','1']
        os.system(' '.join(cmd))

        cmd = ['fslmerge','-t',tmp3,tmp2,tmp1]
        os.system(' '.join(cmd))
        
        os.remove(tmp1 + '.nii.gz')
        os.remove(tmp2 + '.nii.gz')


In [30]:
for scan_tuple in in_tuples:
    sub, exp, run = scan_tuple

    out_dir = os.path.abspath('data/' + sub + '/cal/lyman/')
    out_cal = out_dir + '/fm_' + exp + '_' + str(run) + '_mc_pe0_pe1.nii.gz'
    
    if not os.path.exists(out_cal):
        print scan_tuple



('fd_115', 'loc', 2)
('fd_127', 'ser', 1)
('fd_127', 'sim', 1)
('fd_133', 'sim', 2)
